In [32]:
# NLTK Portion based on
# Based on https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk
# By By Shaumik Daityari, September 26, 2019
#
# Run this if you don't have twitter_samples
import nltk
nltk.download('twitter_samples')
# Run this if you don't have vader_lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/hindle1/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hindle1/nltk_data...


True

In [3]:
import nltk
from nltk.corpus import twitter_samples

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [6]:
print(positive_tweets[0:10])
print(negative_tweets[0:10])

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)', '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!', '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!', '@97sides CONGRATS :)', 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days', '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM', "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI", '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.', 'Jgh , but we have to go to Bayan :D bye', 'As an act of mischievousness, am calling the ETL layer of our in-house warehousing app Katam

In [10]:
import gensim
from gensim.parsing.preprocessing import preprocess_documents
positives = preprocess_documents( positive_tweets )
negatives = preprocess_documents( negative_tweets )

In [11]:
print(positives[0:10])
print(negatives[0:10])

[['followfridai', 'franc', 'int', 'pkuchli', 'milipol', 'pari', 'engag', 'member', 'commun', 'week'], ['lambja', 'hei', 'jame', 'odd', 'contact', 'centr', 'abl', 'assist', 'thank'], ['despiteoffici', 'listen', 'night', 'bleed', 'amaz', 'track', 'scotland'], ['side', 'congrat'], ['yeaaaah', 'yippppi', 'accnt', 'verifi', 'rqst', 'succe', 'got', 'blue', 'tick', 'mark', 'profil', 'dai'], ['bhaktisbant', 'pallaviruhail', 'irresist', 'flipkartfashionfridai', 'http', 'ebzlvenm'], ['like', 'love', 'custom', 'wait', 'long', 'hope', 'enjoi', 'happi', 'fridai', 'lwwf', 'http', 'smyyriipxi'], ['impatientraid', 'second', 'thought', 'there’', 'time', 'new', 'short', 'enter', 'sheep', 'bui'], ['jgh', 'bayan', 'bye'], ['act', 'mischiev', 'call', 'etl', 'layer', 'hous', 'wareh', 'app', 'katamari', 'well…', 'impli']]
[['hopeless', 'tmr'], ['kid', 'section', 'ikea', 'cute', 'shame', 'nearli', 'month'], ['hegelbon', 'heart', 'slide', 'wast', 'basket'], ['ketchburn', 'hate', 'japanes', 'bani'], ['dang', 's

In [23]:
from gensim.corpora import Dictionary
words = Dictionary(positives + negatives)
MAXWORDS=30000
words.filter_extremes(no_below=5, no_above=0.5, keep_n=MAXWORDS)
positive_docs = [words.doc2bow(doc) for doc in positives]
negative_docs = [words.doc2bow(doc) for doc in negatives]



In [24]:
print(positive_docs[0:10])
print(negative_docs[0:10])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(13, 1), (14, 1), (15, 1), (16, 1)], [(17, 1)], [(18, 1), (19, 1), (20, 1), (21, 1)], [(22, 1), (23, 1), (24, 1), (25, 1)], [(24, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [(35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1)], [(42, 1), (43, 1)], [(44, 1), (45, 1), (46, 1), (47, 1)]]
[[], [(152, 1), (180, 1), (619, 1), (630, 1), (1204, 1), (1298, 1)], [(554, 1), (712, 1)], [(545, 1), (703, 1)], [(6, 1), (73, 1), (259, 1)], [(24, 1), (179, 1), (181, 1), (1255, 1)], [(302, 1)], [(24, 1), (73, 1), (475, 1), (545, 1), (751, 1), (830, 1)], [(24, 1)], [(24, 1)]]


In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from gensim.matutils import sparse2full

X = [sparse2full(x,length=MAXWORDS) for x in positive_docs + negative_docs]
y = ["P" for x in positive_docs] + \
    ["N" for x in negative_docs]

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33)


nb = MultinomialNB()
nb.fit(X_train, y_train)
y_hat_train = nb.predict(X_train)
y_hat_test  = nb.predict(X_test)

In [30]:
from sklearn.metrics import confusion_matrix, f1_score

print(confusion_matrix(y_train, y_hat_train))
print(f1_score(y_train, y_hat_train, pos_label="P"))
print(confusion_matrix(y_test, y_hat_test))
print(f1_score(y_test, y_hat_test, pos_label="P"))


[[2678  685]
 [ 685 2652]]
0.7947258016182199
[[1232  405]
 [ 468 1195]]
0.7324547961998161


In [39]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
scores = [sia.polarity_scores(x) for x in positive_tweets + negative_tweets]
print(scores[0:10])
pn_scores = ["N" if x['neg'] > x['pos'] else "P" for x in scores]
print(f1_score(y, pn_scores,pos_label="P"))


[{'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 'compound': 0.7579}, {'neg': 0.145, 'neu': 0.585, 'pos': 0.27, 'compound': 0.6229}, {'neg': 0.0, 'neu': 0.706, 'pos': 0.294, 'compound': 0.7959}, {'neg': 0.0, 'neu': 0.123, 'pos': 0.877, 'compound': 0.7983}, {'neg': 0.0, 'neu': 0.718, 'pos': 0.282, 'compound': 0.795}, {'neg': 0.0, 'neu': 0.565, 'pos': 0.435, 'compound': 0.6597}, {'neg': 0.063, 'neu': 0.417, 'pos': 0.52, 'compound': 0.9466}, {'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compound': 0.4588}, {'neg': 0.0, 'neu': 0.619, 'pos': 0.381, 'compound': 0.7615}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
0.8531605113636365


In [ ]:
from SentiCR import  SentiCR

sentiment_analyzer=SentiCR()

#All examples are acutal code review comments from Go lang

sentences=["I'm not sure I entirely understand what you are saying. "+\
           "However, looking at file_linux_test.go I'm pretty sure an interface type would be easier for people to use.",
           "I think it always returns it as 0.",
           "If the steal does not commit, there's no need to clean up _p_'s runq. If it doesn't commit,"+\
             " runqsteal just won't update runqtail, so it won't matter what's in _p_.runq.",
           "Please change the subject: s:internal/syscall/windows:internal/syscall/windows/registry:",
           "I don't think the name Sockaddr is a good choice here, since it means something very different in "+\
           "the C world.  What do you think of SocketConnAddr instead?",
           "could we use sed here? "+\
            " https://go-review.googlesource.com/#/c/10112/1/src/syscall/mkall.sh "+\
            " it will make the location of the build tag consistent across files (always before the package statement).",
           "Is the implementation hiding here important? This would be simpler still as: "+\
          " typedef struct GoSeq {   uint8_t *buf;   size_t off;   size_t len;   size_t cap; } GoSeq;",
           "Make sure you test both ways, or a bug that made it always return false would cause the test to pass. "+\
        " assertTrue(Testpkg.Negate(false)); "+\
        " assertFalse(Testpkg.Negate(true)); +"\
        " If you want to use the assertEquals form, be sure the message makes clear what actually happened and " +\
        "what was expected (e.g. Negate(true) != false). "]

for sent in sentences:
    score=sentiment_analyzer.get_sentiment_polarity(sent)
    print(sent+"\n Score: "+str(score))


Reading data from oracle..
Training classifier model..


In [9]:
sentiment_analyzer=SentiCR()

#All examples are acutal code review comments from Go lang

sentences=["I'm not sure I entirely understand what you are saying. "+\
           "However, looking at file_linux_test.go I'm pretty sure an interface type would be easier for people to use.",
           "I think it always returns it as 0.",
           "If the steal does not commit, there's no need to clean up _p_'s runq. If it doesn't commit,"+\
             " runqsteal just won't update runqtail, so it won't matter what's in _p_.runq.",
           "Please change the subject: s:internal/syscall/windows:internal/syscall/windows/registry:",
           "I don't think the name Sockaddr is a good choice here, since it means something very different in "+\
           "the C world.  What do you think of SocketConnAddr instead?",
           "could we use sed here? "+\
            " https://go-review.googlesource.com/#/c/10112/1/src/syscall/mkall.sh "+\
            " it will make the location of the build tag consistent across files (always before the package statement).",
           "Is the implementation hiding here important? This would be simpler still as: "+\
          " typedef struct GoSeq {   uint8_t *buf;   size_t off;   size_t len;   size_t cap; } GoSeq;",
           "Make sure you test both ways, or a bug that made it always return false would cause the test to pass. "+\
        " assertTrue(Testpkg.Negate(false)); "+\
        " assertFalse(Testpkg.Negate(true)); +"\
        " If you want to use the assertEquals form, be sure the message makes clear what actually happened and " +\
        "what was expected (e.g. Negate(true) != false). "]

for sent in sentences:
    score=sentiment_analyzer.get_sentiment_polarity(sent)
    print(sent+"\n Score: "+str(score))


TypeError: 'module' object is not callable